#### Introduction

The purpose of this notebook, along with 01_data_setup_example.ipynb and 02_run_example.ipynb is to provide a tutorial of how you may want to use the PopExp pacakge functions.

Please see 01_data_setup_example.ipynb and 02_run_example.ipynb before you work through this notebook!

This notebook is going to explore the results returned by functions in PopExp that were run the previous section of the tutorial. 

In the previous section, we found the number of people affected by any US wildfire disaster in 2016, 2017, and 2018, as well as the number of people affected by any wildfire disaster by ZCTA and each wildfire disaster by ZCTA.

In this section we'll explore the results from each of these function runs.

The first function run helped us find the total number of people affected by any wildfire disaster in 2016, 2017, and 2018. To use these results, we'll first read in and plot the original wildfire disaster dataset, and then read in the results and calculate the total number of people affected by any wildfire disaster.

We'll start by loading libraries and reading in necessary data. 

In [ ]:
import geopandas as gpd 
import pandas as pd
import pathlib
import glob

# some plotting packages for the plots we'll make
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.patches import Circle
from matplotlib.colors import Normalize

We'll read in ZCTA data since the last three PopEx function runs involved ZCTAs, and we'll plot the wildfire disaster data over the California ZCTAs so we can get an idea of what the dataset looks like. We ran these functions to calculate national numbers, but we'll plot our exposure and results in California since it's smaller and we can see what's going on a bit better, since we're demonstrating how these functions work.

In [ ]:
# Define the base path and data directory
base_path = pathlib.Path.cwd().parent
data_dir = base_path / "demo_data"

# Read the raw ZCTA data
zctas = gpd.read_file(data_dir / "01_raw_data" / "tl_2020_us_zcta520" / "tl_2020_us_zcta520.shp")

# Filter ZCTAs for California ZIP codes (90000 to 96100)
zctas_ca = zctas[zctas['GEOID20'].str[:3].astype(int).between(900, 961)]

# Transform to best CRS for plotting CA
teale_albers_crs = "EPSG:3310"
zctas_ca = zctas_ca.to_crs(teale_albers_crs)

In [ ]:
# Read in raw wildfire dataset
fires = gpd.read_file(data_dir / "01_raw_data"/ "wildfires_conus.geojson")

# Filter to wildfires in California that occurred between 2016 and 2018 (inclusive)
fires_ca = fires[(fires['wildfire_states'].str.contains('CA')) & 
                 (fires['wildfire_year'] >= 2016) & 
                 (fires['wildfire_year'] <= 2018)]

# Transform to best CRS for plotting California
fires_ca = fires_ca.to_crs(teale_albers_crs)

First, again just to get an idea of our exposure that we used in the first four function runs, we'll plot all the wildfire disasters in 2016-2018 overlayed on ZCTAs. 

In [ ]:
# Plot the fires overlayed onto ZCTA boundaries

# Plot the ZCTA boundaries first
fig, ax = plt.subplots(figsize=(10, 10))
zctas_ca.boundary.plot(ax=ax, linewidth=0.5, edgecolor='black', zorder=1)

# Overlay the fire geometries with red fill
fires_ca.plot(ax=ax, color='red', alpha=0.5, edgecolor='red', zorder=2)

# Set plot title and labels
ax.set_title('Wildfire disaster boundaries in CA 2016-2018 on CA ZCTAs')
ax.set_axis_off()

# Saving for use in the paper describing the PopExp package
output_path = data_dir / "03_results" / "wildfire_zcta_plot.pdf"
plt.savefig(output_path, format='pdf', bbox_inches='tight')

Nice, ok. 

Again, in the previous section of this tutorial, we calculated five things. We'll start by exploring the results of the the first one: in the first run of find_num_people_affected, we wanted to know the total people residing within 10 km of any wildfire disaster in the US the years 2016, 2017, and 2018. Let's read in those results.

If a user ran find_num_people_affected this way, they'd might be interested in the total number of people affected by wildfire disasters in each year, so to explore the results of this run, we'll calculate those numbers. We'll sum over the hazard IDs. In these results, the hazard IDs might not be unique and might be concatenated for overlapping hazards because we didn't calculate the number of people affected by each unique hazard, rather, we calculated the number of people affected by any hazard.

In [ ]:
# Read output of find_num_people_affected 
tot_af_any_wf = pd.read_parquet(data_dir / "03_results" / "num_people_affected_by_wildfire.parquet")
tot_af_any_wf.head()

Note the column names, types, etc., and how hazard IDs might be concatenated. 

In [ ]:
# Group by year, and sum over hazards for the number of people affected within each year
tot_af_any_wf_grouped = tot_af_any_wf.groupby('year')['num_people_affected'].sum().reset_index()

# Round the output since half a person doesn't make sense
tot_af_any_wf_grouped['num_people_affected'] = tot_af_any_wf_grouped['num_people_affected'].round()

# Show results 
tot_af_any_wf_grouped.head()

That's it, we got the results we wanted for the first run. 

Moving on to the second run. 

In this run, we calculated the total number of people residing within 10km of each unique disaster in each year, by running find_num_people_affected with the parameter by_unique_hazard = True. Someone might have run find_num_people_affected this way if they wanted to identify the most 5 most impactful wildfire disasters in each year, where the boundaries were closest to the largest residential population. To explore the results of this run, we'll find those worst 5 disasters and plot them. 

In [ ]:
# Read in output from find_num_people_affected
tot_unique_wf = pd.read_parquet(data_dir / "03_results" / "num_aff_by_unique_wildfire.parquet")
tot_unique_wf.head()

Note the column names, types again. 

In [ ]:
# Want to group this data by year and find the 5 largest wildfires in each year
# Sort the DataFrame by 'year' and 'num_people_affected' 
sorted_wfs = tot_unique_wf.sort_values(by=['year', 'num_people_affected'], ascending=[True, False])

# Group by 'year' and get the top 5
high_impact_wfs = sorted_wfs.groupby('year').head(5).reset_index(drop=True)
high_impact_wfs

Now we want to plot these. Because find_num_people_affected doesn't preserve the geometry column, we need to rejoin that information to the results. 

In [ ]:
# Prepare to join by correcting col names
fires_ca = fires_ca.rename(columns={'wildfire_id': 'ID_climate_hazard'})
fires_ca = fires_ca[['ID_climate_hazard', 'geometry']].copy()
# And join to get the geographic locations
high_impact_wfs = high_impact_wfs.merge(fires_ca, on='ID_climate_hazard', how='left')


Now let's plot these most impactful disasters on top of the Califonia ZCTAs, so we can see where they are. Let's add some circles proportional to the number of people that were residing within 10 km of each wildfire disaster. One of the most impactful disasters was in Washington State, so we're not going to see that one on this plot. 

We're going to need to work for this plot a little bit. 

In [ ]:
# Get the coordinates of each disaster
high_impact_wfs = high_impact_wfs.set_geometry('geometry')
high_impact_wfs = high_impact_wfs.to_crs(epsg=3310) # reproj 
high_impact_wfs['latitude'] = high_impact_wfs['geometry'].centroid.y
high_impact_wfs['longitude'] = high_impact_wfs['geometry'].centroid.x

# Create df of those coordinates
gdf = gpd.GeoDataFrame(high_impact_wfs, geometry=gpd.points_from_xy(high_impact_wfs.longitude, high_impact_wfs.latitude))
gdf['radius'] = high_impact_wfs['num_people_affected'] / 50
gdf['year'] = gdf['year'].astype('category')

In [ ]:
# Set up to plot by specifying the colors we want to use for each year 
colors = ['green', 'red', 'blue']
unique_years = gdf['year'].cat.categories
color_dict = {year: colors[i % len(colors)] for i, year in enumerate(unique_years)}

# And plot. 
# Plot the ZCTA boundaries first
fig, ax = plt.subplots(figsize=(10, 10))
zctas_ca.boundary.plot(ax=ax, linewidth=0.5, edgecolor='black', zorder=1)

# Plot the wildfire coordinates and a circle proportional to the number of people affected, colored by year
for idx, row in gdf.iterrows():
    color = color_dict[row['year']]
    ax.plot(row.geometry.x, row.geometry.y, 'o', color=color, markersize=5, zorder=2)
    circle = Circle((row.geometry.x, row.geometry.y), row.radius, color=color, fill=True, alpha=0.2, zorder=2)
    ax.add_patch(circle)

# Set plot title and labels
ax.set_title('Five fires with largest population residing within 10km of fire boundary, by year 2016-2018')
ax.text(0.5, 0.99, 'Circle size proportional to number of people affected', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
ax.set_axis_off()

# Add a legend 
handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color_dict[year], markersize=10, label=str(year)) for year in unique_years]
ax.legend(handles=handles, title='Year', loc='upper right')

# Create an inset map for LA area with adjusted position
ax_inset = inset_axes(ax, width="30%", height="30%", loc='lower left', bbox_to_anchor=(-0.10, 0.05, 1, 1), bbox_transform=ax.transAxes, borderpad=2)
la_zctas = zctas_ca[zctas_ca['GEOID20'].astype(int).between(90001, 91699)]
zctas_ca.boundary.plot(ax=ax_inset, linewidth=0.5, edgecolor='black')

# Plot the wildfire coordinates and cirlces colored by year in the inset map
for idx, row in gdf.iterrows():
    if row.geometry.within(la_zctas.unary_union):
        color = color_dict[row['year']]
        ax_inset.plot(row.geometry.x, row.geometry.y, 'o', color=color, markersize=5, zorder=2)
        circle = Circle((row.geometry.x, row.geometry.y), row.radius, color=color, fill=True, alpha=0.2, zorder=2)
        ax_inset.add_patch(circle)

# Set the extent of the inset map to the bounds of the LA ZCTAs
xmin, ymin, xmax, ymax = la_zctas.total_bounds
ax_inset.set_xlim(xmin, xmax)
ax_inset.set_ylim(ymin, ymax)

ax_inset.set_title('LA Area')
ax_inset.set_axis_off()

Lauren if you have feedback on how to organize and run that plot more nicely pls share! I always get skeeved out by how messy plotting code can look and I don't have good organization ideas for it! 

Ok - that's what we wanted from the second run.

Now, let's visualize the results of the third and fourth demonstrations we did, where we ran 'find_num_people_affected_by_geo'. In these cases, we found the number of people who resided within 10 km of any wildfire disaster by year and by ZCTA, and the number of people who resided within 10 km of each wildfire disaster by ZCTA. 

Why would someone want the number of people who resided within 10 km of any wildfire disaster by year and by ZCTA? A researcher may have wanted to find the number of people affected by any wildfire by ZCTA if they wanted to assess wildfire disaster exposure by ZCTA. They might want to know what proportion of people in each ZCTA lived within 10 km of any disaster boundary and were therefore exposed to fire, and then consider a ZCTA exposed if enough of its population was exposed. If we were doing that exposure assessment, we'd probably want to plot the proportion of people exposed to disasters by ZCTA. So, let's use our results to do that. To accomplish that, we also need to use the denominator data that we produced at the end of the last section, where we found the ZCTA-level population. 

Let's start by reading in that denominator data, and plotting the number of people who live in each ZCTA.

In [ ]:
# Read results showing the number of people residing in each ZCTA
num_residing_by_zcta = pd.read_parquet(data_dir / "03_results" / "num_people_residing_by_zcta.parquet")

# CA only based on ZCTAs
num_residing_ca = num_residing_by_zcta[pd.to_numeric(num_residing_by_zcta['ID_spatial_unit']).between(90000, 96100)].copy()

# select cols ID spatial unit and num_people_affected
num_residing_ca = num_residing_ca[["ID_spatial_unit", "num_people_residing"]]
num_residing_ca.head()

In [ ]:
# Clean zctas for plotting
zctas_ca.rename(columns={"ZCTA5CE20": "ID_spatial_unit"}, inplace=True)
zctas_ca = zctas_ca[["ID_spatial_unit", "geometry"]]

# Merge population data to zctas_ca geometry for plotting
zctas_ca = zctas_ca.merge(num_residing_ca, on="ID_spatial_unit", how="left")

In [ ]:
# Find LA and Bay ZCTAs for map insets
la_zctas = zctas_ca[zctas_ca['ID_spatial_unit'].astype(int).between(90000, 91610)]
sf_zctas = zctas_ca[zctas_ca['ID_spatial_unit'].astype(int).between(94000, 94199)]

# Plot.
fig, ax = plt.subplots(figsize=(10, 10))

# Create the main plot WITHOUT the legend parameter
zctas_ca.plot(column='num_people_residing', ax=ax, cmap='viridis', 
              linewidth=0.1, edgecolor='black')

# Create a colorbar as the legend
norm = Normalize(vmin=zctas_ca['num_people_residing'].min(), vmax=zctas_ca['num_people_residing'].max())
sm = plt.cm.ScalarMappable(cmap='viridis', norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, fraction=0.03, pad=0.04)
cbar.set_label('Number of People Residing')

# Set plot title and labels
ax.set_title('Population by 2020 ZCTA according to GHSL 2020 100m\n resolution gridded population dataset', pad=20)
ax.set_axis_off()

# Create an inset map for LA area with adjusted position
ax_inset_la = inset_axes(ax, width="30%", height="30%", loc='lower left', 
                         bbox_to_anchor=(-0.4, 0.05, 1, 1), 
                         bbox_transform=ax.transAxes, borderpad=2)
zctas_ca.plot(column='num_people_residing', ax=ax_inset_la, cmap='viridis', 
              linewidth=0.1, edgecolor='black')

# Set the extent of the inset map to the bounds of the LA ZCTAs
xmin, ymin, xmax, ymax = la_zctas.total_bounds
ax_inset_la.set_xlim(xmin, xmax)
ax_inset_la.set_ylim(ymin, ymax)
ax_inset_la.set_title('LA Area')
ax_inset_la.set_axis_off()

# Create an inset map for SF area with adjusted position
ax_inset_sf = inset_axes(ax, width="30%", height="30%", loc='lower left', 
                         bbox_to_anchor=(-0.4, 0.45, 1, 1), 
                         bbox_transform=ax.transAxes, borderpad=2)
zctas_ca.plot(column='num_people_residing', ax=ax_inset_sf, cmap='viridis', 
              linewidth=0.1, edgecolor='black')

# Set the extent of the inset map to the bounds of the SF ZCTAs
xmin, ymin, xmax, ymax = sf_zctas.total_bounds
ax_inset_sf.set_xlim(xmin, xmax)
ax_inset_sf.set_ylim(ymin, ymax)
ax_inset_sf.set_title('Bay Area')
ax_inset_sf.set_axis_off()

# Adjust the layout to make room for the legend
plt.tight_layout(rect=[0, 0, 0.85, 1])  # Adjust the right padding to make room for the legend

# Save fig for use in pop_exp package paper
output_path = data_dir / "03_results" / "pop_by_zcta_plot.png"
plt.savefig(output_path, format='png', bbox_inches='tight', dpi=300)  # Set dpi for higher resolution

Ok so those are our denominators. Now let's get the number of people affected by *any* wildfire by ZCTA. 

In [ ]:
# Read output of find_num_people_affected_by_geo
wf_by_zcta = pd.read_parquet(data_dir / "03_results" / "num_people_affected_by_wildfire_by_zcta.parquet")
wf_by_zcta.head()

In [ ]:
zctas_ca.head()

In [ ]:
# someone actually doing this might want to plot all years, but let's just select 2018
# and plot that
wf_by_zcta_2018 = wf_by_zcta[wf_by_zcta['year'] == 2018].copy()
# drop the year col
wf_by_zcta_2018.drop(columns='year', inplace=True)
wf_by_zcta_2018.head()

In [ ]:
# merge to zctas_ca geometry for plotting
zctas_ca_wf = zctas_ca.merge(wf_by_zcta_2018, on="ID_spatial_unit", how="left")
zctas_ca_wf.head()

In [ ]:
# fill in NAs in num people affected with 0
zctas_ca_wf['num_people_affected'] = zctas_ca_wf['num_people_affected'].fillna(0)
zctas_ca_wf.head()


In [ ]:
# find the proportion of people affected by ZCTA
zctas_ca_wf['prop_aff'] = zctas_ca_wf['num_people_affected'] / zctas_ca_wf['num_people_residing']

In [ ]:
# Plot. 
fig, ax = plt.subplots(figsize=(10, 10))
zctas_ca_wf.plot(column='prop_aff', ax=ax, legend=True, cmap='viridis', linewidth=0.1, edgecolor='black')

# Set plot title and labels
ax.set_title('Proportion of ZCTA population residing within 10km of a 2018\n wildfire boundary by 2020 ZCTA')
ax.set_axis_off()

# Create an inset map for LA area with adjusted position
ax_inset_la = inset_axes(ax, width="30%", height="30%", loc='lower left', bbox_to_anchor=(-0.4, 0.05, 1, 1), bbox_transform=ax.transAxes, borderpad=2)
zctas_ca_wf.plot(column='prop_aff', ax=ax_inset_la, cmap='viridis', linewidth=0.1, edgecolor='black', legend=False)

# Set the extent of the inset map to the bounds of the LA ZCTAs
xmin, ymin, xmax, ymax = la_zctas.total_bounds
ax_inset_la.set_xlim(xmin, xmax)
ax_inset_la.set_ylim(ymin, ymax)

ax_inset_la.set_title('LA Area')
ax_inset_la.set_axis_off()

# Create an inset map for SF area with adjusted position
ax_inset_sf = inset_axes(ax, width="30%", height="30%", loc='lower left', bbox_to_anchor=(-0.4, 0.45, 1, 1), bbox_transform=ax.transAxes, borderpad=2)
zctas_ca_wf.plot(column='prop_aff', ax=ax_inset_sf, cmap='viridis', linewidth=0.1, edgecolor='black', legend=False)

# Set the extent of the inset map to the bounds of the SF ZCTAs
xmin, ymin, xmax, ymax = sf_zctas.total_bounds
ax_inset_sf.set_xlim(xmin, xmax)
ax_inset_sf.set_ylim(ymin, ymax)

ax_inset_sf.set_title('Bay Area')
ax_inset_sf.set_axis_off()

# Save output path
output_path = data_dir / "03_results" / "prop_pop_by_wf_by_zcta_plot.png"
plt.savefig(output_path, format='png', bbox_inches='tight', dpi=300)  # Set dpi for higher resolution

Ok, great. 

One final thing about this output - this output shows us the proprtion of the ZCTA population residing within 10km of a wildfire disaster boundary by ZCTA. 

If instead of the total population, we wanted to find the total number of people making >100,000$ in household income living within 10km of a wildfire disaster boundary by ZCTA, or the total number children under the age of 5 residing within 10km of a wildfire disaster boundary by ZCTA, we could use census data together with this function output to do this. We could do this for any census-described population. If we assume that the population of interest (ex. children under 5) is uniformly distributed throughout the residential population, we can take the function output describing the proportion of the population residing within 10km of the fire boundaries and multiply this by the under 5 population to estimate the number of children exposed. This is improves upon the strategy of using census data and hazard boundaries to find the exposed populuation, which assumes that the population is uniformly distributed across space. 

Finally, let's look at the results from finding the number of people affected by each unique wildfire disaster in each ZCTA, the fourth quantity we calculated in the previous section of the tutorial. There are many reasons why someone might want to calculate this type of exposure - maybe they want to look at multiple exposures. To explore these results, we'll keep it simple. A researcher might be interested in this option if they wanted to know about multiple exposures in the same ZCTA or a group of ZCTAs, maybe for an interrupted time series analysis or something similar. If that was the goal, they might want to look at exposure over time in a specific ZCTA. They also might want to know which ZCTAs were affected by a certain disaster.
 
First, let's filter the results to look at ZCTA 90263, downtown Malibu.

In [ ]:
# need to read in results 
unique_wf_by_zcta = pd.read_parquet(data_dir / "03_results" / "num_people_affected_by_wildfire_zcta_unique.parquet")
unique_wf_by_zcta.head()

In [ ]:
# filter to zcta 90263
unique_wf_by_zcta_90263 = unique_wf_by_zcta[unique_wf_by_zcta['ID_spatial_unit'] == '90263']
print(unique_wf_by_zcta_90263)

We can see that the Woolsey fire boundary was within 10km of 1578 people living in this ZCTA. The fire burned some structures and homes in Malibu, but did not burn through the whole downtown, so some of these people might have been living in the fire boundary - we'd need to map the raster and fire boundary data to tell. 

Let's also look at the Camp Fire. 

In [ ]:
unique_wf_by_zcta_cf = unique_wf_by_zcta[unique_wf_by_zcta['ID_climate_hazard'] == '2018-11-08_CAMP_CA_BUTTE_5168']
print(unique_wf_by_zcta_cf)

Looks like there were people in 15 different ZCTAs who were living within 10km of the fire boundary or closer - in this case we know this fire burned completely through Paradise so a lot of thses people were actually living inside the fire boundary rather than just close to it. 

We already used the output from the fifth function to calculate the proportion of the ZCTA populations affected by wildfire disasters, so that's the end. We hope you find these functions useful for population-level environmental exposure assignment. 